기본 세팅

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!pip install konlpy
!pip install kss

depression score model

In [42]:
import pandas as pd
import numpy as np
import kss
import re
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import KFold

def convert_to_ord(data):
    try:
        return [ord(xx) for xx in data]
    except:
        print(data)

df_blog = pd.read_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/blog text data.csv')
df = []

model_yn = load_model('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/model_yn.h5')
model_09 = load_model('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/model_09.h5')

for i in range(len(df_blog)):
    sentence_list = kss.split_sentences(df_blog.iloc[i,3])
    for j in range(len(sentence_list)):
        df.append({'이름':df_blog.iloc[i,0], '날짜':df_blog.iloc[i,1], '제목':df_blog.iloc[i,2], '내용':' '.join(re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]', ' ', str(sentence_list[j]).strip()).split()),'ord':np.nan,'label_1':np.nan,'label_2':np.nan})

df = pd.DataFrame(df)

df['ord'] = df['내용'].map(convert_to_ord)

max_len = 64

sentence_ord = sequence.pad_sequences(df['ord'], maxlen = max_len)
sentence_ord = np.array(sentence_ord).reshape((len(sentence_ord), max_len))

df['label_1'] = model_yn.predict_classes(sentence_ord)
df['label_2'] = model_09.predict_classes(sentence_ord)

df['label_2'][df['label_1']==0] = np.nan

df_yn = df.groupby(['이름','label_1']).count()
df_yn = df_yn.reset_index()
df_yn = df_yn.pivot(index='이름',columns='label_1',values='내용')
df_yn.columns = ['N','Y']
df_yn['all'] = df_yn['N']+df_yn['Y']
df_yn['p'] = df_yn['Y']/(df_yn['N']+df_yn['Y'])
df_yn = df_yn[['all','p','N','Y']]

df_09 = df.groupby(['이름','label_1','label_2']).count()
df_09 = df_09.reset_index()
df_09 = df_09.pivot(index='이름',columns='label_2',values='내용')
df_09.columns = ['0','1','2','3','4','5','6','7','8','9']

df_score_board = pd.merge(df_yn, df_09, on='이름')

df_real_score = pd.read_excel('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/list.xlsx', encoding='cp949')

df_real_score = df_real_score.replace('없음', 0)
df_real_score = df_real_score.replace('2~6일', 1)
df_real_score = df_real_score.replace('7~12일', 2)
df_real_score = df_real_score.replace('거의 매일', 3)

df_score_board = pd.merge(df_score_board, df_real_score, on='이름')
df_score_board['base_model'] = np.nan
df_score_board['a'] = np.nan
df_score_board['b,c'] = np.nan
df_score_board = df_score_board.fillna(0)

df_score_board.to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/asdf.csv', encoding='cp949', index=False)

kf = KFold(n_splits=10, shuffle=True)
n = 1
for i,j in kf.split(df_score_board):
    df_score_board.iloc[i].to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_train_'+str(n)+'.csv', encoding='cp949', index=False)
    df_score_board.iloc[j].to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_test_'+str(n)+'.csv', encoding='cp949', index=False)
    n += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


"\nkf = KFold(n_splits=10, shuffle=True)\nn = 1\nfor i,j in kf.split(df_score_board):\n    df_score_board.iloc[i].to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_train_'+str(n)+'.csv', encoding='cp949', index=False)\n    df_score_board.iloc[j].to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_test_'+str(n)+'.csv', encoding='cp949', index=False)\n    n += 1\n"